In [ ]:
# !pip install notify_run
%matplotlib notebook 
%load_ext autoreload
%autoreload 2

# from google.colab import drive
# drive.mount('/content/gdrive')

# Setup

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image

from tqdm import tqdm, tnrange, tqdm_notebook
from notify_run import Notify

In [ ]:
notify = Notify()
# notify.register()

In [ ]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from src import constants as c
from src.model import VAE
from src import visualization as v

# Dataset Initialization

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize(c.image_size),
    transforms.CenterCrop(c.image_size),
    transforms.ToTensor()
])

image_datasets = {x: datasets.ImageFolder(os.path.join(c.data_home, 'surgical_data/',x),
                                          data_transforms)
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                              batch_size=c.batch_size,
                                              shuffle=True)
               for x in ['train', 'val']}

# Training setup

In [ ]:
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE, KLD

In [ ]:
# for zdim in [2,5,10,15]:
for zdim in [2]:

    model = VAE(image_channels=c.image_channels,
                image_size=c.image_size, 
                h_dim1=1024,
                h_dim2=128,
                zdim=zdim).to(c.device)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    for epoch in tqdm_notebook(range(50)):
        notify.send("z-dim = {}, Training Epoch {}".format(zdim, epoch+1))
        
        """
        Training
        """
        model.train()
        train_loss = 0
        for batch_idx, (data, _) in enumerate(tqdm_notebook(dataloaders['train'])):
            data = data.to(c.device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)
            bce, kld = loss_function(recon_batch, data, mu, logvar)
            loss = bce + kld
            loss.backward()
            train_loss += loss.item()
            optimizer.step()

            if batch_idx % c.log_interval == 0:
                tqdm.write('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tKL-Divergence: {:.6f}'.format(
                    epoch,
                    batch_idx * len(data), 
                    len(dataloaders['train'].dataset),
                    100. * batch_idx / len(dataloaders['train']),
                    loss.item() / len(data),
                    kld.item() / len(data)))

        tqdm.write('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, 
            train_loss / len(dataloaders['train'].dataset)))

        """
        Testing
        """
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for i, (data, _) in enumerate(tqdm_notebook(dataloaders['val'])):
                data = data.to(c.device)
                recon_batch, mu, logvar = model(data)
                test_loss += loss_function(recon_batch, data, mu, logvar).item()
                if i == 0:
                    n = min(data.size(0), 8)
                    comparison = torch.cat([data[:n],
                                            recon_batch.view(batch_size, 3, 64, 64)[:n]])

                    save_image(comparison.cpu(),
                               data_home + 'samples/reconstruction_epoch_{}_zdim_{}.png'.format(epoch, zdim), nrow=n)
        
    torch.save(model.state_dict(), c.data_home + "weights/tools_vae_{}_epochs_{}_zdim_{}.torch".format(c.image_size, 
                                                                                                       epochs, 
                                                                                                       zdim))

    with torch.no_grad():
        z = torch.randn(64, zdim)
        sample = model.decode(z.to(c.device))
        save_image(sample.view(64, 
                               c.image_channels, 
                               c.image_size, 
                               c.image_size).cpu(), 
                   c.data_home + 'figures/latent_space_random_{}_epochs_{}_zdim_{}.png'.format(c.image_size, 
                                                                                            epochs, 
                                                                                            zdim))

# Model evaluation

In [ ]:
import pandas as pd
from sklearn import decomposition, manifold
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
models = {zdim: VAE(image_channels=c.image_channels,
                    image_size=c.image_size, 
                    h_dim1=1024,
                    h_dim2=128,
                    zdim=zdim).to(c.device) for zdim in [5]}

In [ ]:
models[5]

In [ ]:
for zdim, model in models.items():
    model.load_state_dict(torch.load(c.data_home + "weights/tools_vae_{}_epoch_50_zdim_{}.torch".format(c.image_size,
                                                                                                        zdim)))

## Saving latent space encodings

In [ ]:
labels = pd.read_csv(os.path.join(c.data_home, 'surgical_data/', 'surgical_labels.csv'))

In [ ]:
encoded_inputs = {zdim: [] for zdim in [5]}

with torch.no_grad():
    for zdim in tqdm_notebook(encoded_inputs):
        for index in tnrange(len(image_datasets['train'])):
            data = image_datasets['train'][index][0].view(-1, c.image_channels, c.image_size, c.image_size).to(c.device)
            latent_vector = models[zdim].sampling(*models[zdim].encode(data)).cpu().detach().numpy()
            encoded_inputs[zdim].extend([ar[0] for ar in np.split(latent_vector, data.shape[0])])
            
        for index in tnrange(len(image_datasets['val'])):
            data = image_datasets['val'][index][0].view(-1, c.image_channels, c.image_size, c.image_size).to(c.device)
            latent_vector = models[zdim].sampling(*models[zdim].encode(data)).cpu().detach().numpy()
            encoded_inputs[zdim].extend([ar[0] for ar in np.split(latent_vector, data.shape[0])])

In [ ]:
latent_space = pd.concat({zdim: pd.concat([pd.DataFrame(encoded_inputs[zdim]), labels], axis=1) for zdim in [5]})
latent_space.head()

In [ ]:
latent_space.to_csv(os.path.join(c.data_home,'latent_space_zdim_5.csv'))

## PCA on latent space

In [ ]:
for zdim in [5]:
    components=5
    pca = decomposition.PCA(n_components=components)
    pca_result = pca.fit_transform(latent_space.loc[zdim][[0,1,2,3,4]].values)
    for i in range(components):
        latent_space.loc[zdim,'pc{}'.format(i+1)] = pca_result[:,i]
    print('z={}:\tExplained variation per principal component: {} {}'.format(zdim, sum(pca.explained_variance_ratio_),pca.explained_variance_ratio_))

In [ ]:
fig = v.plot_pca(5, latent_space.loc[5])
fig.savefig('./pca_zdim_5.png')

In [ ]:
fig, ax = plt.subplots()
v.imscatter(latent_space.loc[5]['pc1'], 
          latent_space.loc[5]['pc2'], 
          data=image_datasets['train'], 
          ax=ax, 
          zoom=0.25
         )

plt.show()

## PCA on Latent space (n=3)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(latent_space.loc[5]['pc1'], 
                     latent_space.loc[5]['pc2'], 
                     latent_space.loc[5]['pc3'], 
                     c=latent_space.loc[5]['Tool'], cmap='brg')
colorbar = fig.colorbar(scatter, ax=ax)
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
plt.title("Scatter Plot of Principal Components of 5-dimensional latent space")
plt.show()

In [ ]:
fig, ax = plt.subplots()
v.imscatter(latent_space.loc[5]['pc1'], 
          latent_space.loc[5]['pc2'], 
          data=image_datasets['train'], 
          ax=ax, 
          zoom=0.25
         )

plt.show()

## T-SNE on latent space

In [ ]:
for zdim in [5]:
    tsne_result = manifold.TSNE(n_components=2).fit_transform(latent_space.loc[zdim][[0,1,2,3,4]])
    latent_space.loc[zdim, 'tsne1'] = tsne_result[:,0]
    latent_space.loc[zdim, 'tsne2'] = tsne_result[:,1]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
scatter = ax.scatter(latent_space.loc[5]['tsne1'], latent_space.loc[5]['tsne2'], c=latent_space.loc[5]['Tool'], cmap='brg')
plt.show()

In [ ]:
latent_space.to_csv(c.data_home + 'latent_space_zdim_5.csv')

In [ ]:
fig, ax = plt.subplots()
v.imscatter(latent_space.loc[5]['tsne1'], 
          latent_space.loc[5]['tsne2'], 
          data=image_datasets['train'], 
          ax=ax, 
          zoom=0.25)

plt.show()